In [1]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob


In [2]:
ps.__version__

'1.3.0'

In [3]:
WEAK = False

x_len = 5000
t_len = 6000

xs = np.arange(x_len)
dt = 1

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]  

if WEAK:
    X, T = np.meshgrid(xs, np.arange(t_len))
    XT = np.asarray([X, T]).T

    pde_lib = ps.WeakPDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatiotemporal_grid=XT,
        is_uniform=True,
        K=1000,
    )
else:
    
    pde_lib = ps.PDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatial_grid=xs,
        include_bias=True,
        is_uniform=True,
    )
    

In [4]:
def printEnsemble(model, cut_off = 1e-3, median= False):
    if median:
        coefs = np.median(model.coef_list, axis=0)[0,:]
    else:
        coefs = np.mean(model.coef_list, axis=0)[0,:]
    above_cut_off = np.argwhere(np.abs(coefs)>cut_off).flatten()
    features =  model.get_feature_names()
    equation_str= "(x0)' = "
    for count, val in enumerate(above_cut_off):
        if count >0:
            equation_str+=" + "
        equation_str+= "{0:.4f}".format(coefs[val])
        equation_str+= " "+features[val]
        
    print(equation_str)

In [5]:
def saveEnsemble(model,fileStr,thresh):
    np.save("/home/ldoyle/notebooks/ensembleSINDyArrays_30_run4/"+fileStr+"_thresh_"+"{:.2E}".format(thresh), model.coef_list)
#     print(fileStr+"_thresh_"+"{:.2E}".format(thresh))

In [5]:
def SINDyImplementEnsemble(dataset, fileStr,thresh, alph=0.001, n_models=30, time_steps=400, stlsq_max =100,median=True):
    optimizer = ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max)
#     optimizer = ps.EnsembleOptimizer( opt=ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max),bagging=True,  n_models = n_models,n_subset =time_steps)  
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, 
                     differentiation_method= ps.differentiation.SmoothedFiniteDifference())
    model.fit(dataset, t=dt,ensemble=True, n_models=n_models, n_subset=time_steps)
    print("Ensemble STLSQ with Threshold "+str(thresh))
    printEnsemble(model,median=median)
#     saveEnsemble(model,fileStr,thresh)

## Unfiltered

### Dataset 1

In [6]:
path = "/data/data2/south-data-ejm/hdd/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/"

fileStr = "2021-11-01T2311_unfiltered"

init = 11
final= 16

time_subsample = 10
full_dat = np.zeros((5000,(12000//time_subsample)*(final-init)))

k = 0
for i in np.arange(init,final):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    
working_dat = full_dat/np.std(full_dat)


In [7]:
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [8]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6464 x0_1 + 0.3595 x0_111


In [9]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6285 x0_1 + 0.3505 x0_111 + 0.0134 x0x0x0_1


In [10]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6340 x0_1 + 0.0148 x0_11 + 0.3466 x0_111 + 0.0033 x0_1111 + -0.0050 x0x0_1 + 0.0110 x0x0x0_1 + -0.0086 x0x0_11 + -0.0055 x0x0x0_11 + -0.0057 x0x0_111 + 0.0048 x0x0x0_111 + -0.0016 x0x0x0_1111


In [11]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6380 x0_1 + 0.0053 x0_11 + 0.3511 x0_111 + 0.0049 x0_1111 + -0.0064 x0x0_1 + 0.0099 x0x0x0_1 + -0.0050 x0x0x0_11 + -0.0103 x0x0_111 + 0.0054 x0x0x0_111


In [12]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6258 x0_1 + 0.3429 x0_111 + 0.0054 x0x0_1 + 0.0147 x0x0x0_1 + -0.0068 x0x0_11 + 0.0077 x0x0x0_111


### Dataset 2

In [13]:
time_subsample = 10
full_dat = np.zeros((5000,6000))
fileStr = "2021-11-01T2321_unfiltered"


k = 0
for i in np.arange(21,26):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    


In [14]:
working_dat = full_dat/np.std(full_dat)#*10**6
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [15]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6682 x0_1 + 0.3816 x0_111


In [16]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6782 x0_1 + 0.3846 x0_111


In [17]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.0024 x0 + 0.0018 x0x0 + 0.6693 x0_1 + 0.3838 x0_111 + 0.0039 x0x0_1 + 0.0063 x0x0x0_1 + 0.0064 x0x0_11 + -0.0123 x0x0_111 + 0.0032 x0x0x0_111 + 0.0026 x0x0_1111 + -0.0015 x0x0x0_1111


In [18]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6624 x0_1 + -0.0048 x0_11 + 0.3829 x0_111 + -0.0066 x0x0_1 + -0.0055 x0x0_11 + -0.0170 x0x0_111


In [19]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6674 x0_1 + 0.3748 x0_111 + 0.0067 x0x0x0_111


## Drift Removal Load

### Dataset 1

In [6]:
fileStr = "2021-11-01T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
working_dat.shape

(5000, 6000, 1)

In [13]:
thresh = 3.4e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity

Ensemble STLSQ with Threshold 0.34
(x0)' = 


/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.34) and eliminated all coefficients
  warnings.warn(


In [21]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6865 x0_1 + 0.3235 x0_111


In [22]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.7044 x0_1 + 0.3334 x0_111


In [23]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.7024 x0_1 + 0.3347 x0_111 + -0.0150 x0x0x0_1 + -0.0065 x0x0x0_111


In [7]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.7021 x0_1 + 0.0029 x0_11 + 0.3283 x0_111 + -0.0092 x0x0_1 + -0.0103 x0x0x0_1 + 0.0048 x0x0_11 + -0.0073 x0x0x0_11 + 0.0019 x0x0_111 + -0.0061 x0x0x0_111


In [25]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.7218 x0_1 + -0.0073 x0_11 + 0.3467 x0_111 + -0.0030 x0_1111 + 0.0012 x0x0_1 + -0.0205 x0x0x0_1 + -0.0075 x0x0_11 + -0.0128 x0x0x0_111 + -0.0024 x0x0_1111


### Dataset 2

In [8]:
fileStr = "2021-11-01T2321_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [27]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6570 x0_1 + 0.3089 x0_111


In [28]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6825 x0_1 + 0.3204 x0_111


In [29]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6703 x0_1 + 0.3235 x0_111 + -0.0109 x0x0_1


In [9]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6844 x0_1 + 0.3123 x0_111 + -0.0020 x0_1111 + -0.0072 x0x0x0_1 + -0.0072 x0x0_11


In [31]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6703 x0_1 + -0.0082 x0_11 + 0.3131 x0_111 + 0.0020 x0x0_1 + 0.0105 x0x0x0_1 + -0.0137 x0x0_11 + -0.0042 x0x0x0_11 + -0.0056 x0x0_111 + 0.0087 x0x0x0_111 + -0.0039 x0x0_1111


### Dataset 3

In [10]:
fileStr = "2021-11-02T0511_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [33]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6833 x0_1 + 0.3208 x0_111


In [34]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6788 x0_1 + 0.3229 x0_111


In [35]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6946 x0_1 + 0.3302 x0_111 + -0.0155 x0x0x0_1


In [11]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.7055 x0_1 + 0.0108 x0_11 + 0.3352 x0_111 + 0.0107 x0x0_1 + -0.0167 x0x0x0_1 + 0.0087 x0x0_111 + -0.0050 x0x0x0_111


In [37]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6881 x0_1 + -0.0075 x0_11 + 0.3260 x0_111 + 0.0018 x0x0_1 + -0.0077 x0x0x0_1 + -0.0087 x0x0_11 + 0.0101 x0x0x0_11 + -0.0025 x0x0x0_111 + -0.0040 x0x0_1111 + 0.0021 x0x0x0_1111


### Dataset 4

In [46]:
fileStr = "2021-11-02T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [39]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6135 x0_1 + 0.2843 x0_111


In [40]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6263 x0_1 + 0.2939 x0_111


In [41]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6301 x0_1 + 0.2984 x0_111 + -0.0071 x0x0x0_1 + -0.0031 x0x0x0_111


In [13]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6280 x0_1 + -0.0083 x0_11 + 0.3049 x0_111 + -0.0019 x0x0_1 + -0.0049 x0x0_111


In [43]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6362 x0_1 + -0.0013 x0_11 + 0.2929 x0_111 + 0.0049 x0x0_1 + -0.0058 x0x0x0_1 + 0.0029 x0x0_11 + -0.0034 x0x0x0_11 + 0.0062 x0x0_111 + -0.0043 x0x0x0_111 + 0.0020 x0x0_1111


## Multifilt

### Dataset 1

In [7]:
fileStr = "2021-11-01T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [45]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9962 x0_1


In [8]:
thresh = 8e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.08
(x0)' = 0.9964 x0_1 + 0.1028 x0_111


In [46]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9970 x0_1 + 0.0873 x0_111 + 0.0639 x0x0x0_111


In [20]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9968 x0_1 + 0.0838 x0_111 + 0.0614 x0x0x0_111


In [48]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9968 x0_1 + 0.0830 x0_111 + 0.0636 x0x0x0_111


In [49]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9952 x0_1 + 0.0863 x0_111 + 0.0018 x0x0x0_1 + -0.0022 x0x0_111 + 0.0687 x0x0x0_111


In [50]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9950 x0_1 + -0.0027 x0_11 + 0.0838 x0_111 + 0.0019 x0x0x0_1 + -0.0029 x0x0_111 + 0.0692 x0x0x0_111


### Dataset 2

In [9]:
fileStr = "2021-11-01T2321_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [8]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9963 x0_1


In [10]:
thresh = 8e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.08
(x0)' = 0.9967 x0_1 + 0.1205 x0_111


In [28]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9972 x0_1 + 0.0992 x0_111 + 0.0609 x0x0x0_111


In [22]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9970 x0_1 + 0.0944 x0_111 + 0.0605 x0x0x0_111


In [14]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9972 x0_1 + 0.0971 x0_111 + 0.0625 x0x0x0_111


In [8]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9957 x0_1 + 0.0955 x0_111 + 0.0015 x0x0x0_1 + 0.0703 x0x0x0_111


In [9]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9956 x0_1 + 0.0985 x0_111 + 0.0016 x0x0x0_1 + -0.0010 x0x0_111 + 0.0688 x0x0x0_111


In [34]:
thresh = 8e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.8
(x0)' = 0.9857 x0_1


In [35]:
thresh = 5e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.5
(x0)' = 0.9901 x0_1 + 0.7876 x0_111


In [36]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9913 x0_1 + 0.2486 x0_11 + 0.6362 x0_111 + 0.4078 x0x0x0_111


In [37]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9920 x0_1 + 0.2425 x0_11 + 0.6415 x0_111 + 0.4103 x0x0x0_111


In [38]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9916 x0_1 + 0.1872 x0_11 + 0.6255 x0_111 + 0.0210 x0_1111 + 0.0458 x0x0x0_11 + 0.4102 x0x0x0_111 + 0.0524 x0x0x0_1111


### Dataset 3

In [11]:
fileStr = "2021-11-02T0511_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [11]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9967 x0_1


In [12]:
thresh = 8e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.08
(x0)' = 0.9967 x0_1 + 0.1249 x0_111


In [30]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9970 x0_1 + 0.1036 x0_111 + 0.0536 x0x0x0_111


In [24]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9971 x0_1 + 0.1017 x0_111 + 0.0564 x0x0x0_111


In [16]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9974 x0_1 + 0.1028 x0_111 + 0.0601 x0x0x0_111


In [10]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9957 x0_1 + 0.0987 x0_111 + 0.0014 x0x0x0_1 + -0.0013 x0x0_111 + 0.0655 x0x0x0_111


In [12]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9956 x0_1 + 0.1011 x0_111 + 0.0013 x0x0x0_1 + 0.0018 x0x0_111 + 0.0618 x0x0x0_111 + -0.0018 x0x0x0_1111


In [40]:
thresh = 8e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.8
(x0)' = 0.9842 x0_1


In [41]:
thresh = 5e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.5
(x0)' = 0.9885 x0_1 + 0.7933 x0_111


In [42]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9915 x0_1 + 0.2463 x0_11 + 0.6791 x0_111 + 0.3621 x0x0x0_111


In [43]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9914 x0_1 + 0.2450 x0_11 + 0.6783 x0_111 + 0.3874 x0x0x0_111


In [44]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9921 x0_1 + 0.2116 x0_11 + 0.6587 x0_111 + 0.0328 x0x0x0_11 + 0.3925 x0x0x0_111 + 0.0435 x0x0x0_1111


### Dataset 4

In [13]:
fileStr = "2021-11-02T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [14]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9978 x0_1 + 0.2204 x0_111


In [14]:
thresh = 8e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.08
(x0)' = 0.9978 x0_1 + 0.2152 x0_111


In [32]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9983 x0_1 + 0.1798 x0_111 + 0.0607 x0x0x0_111


In [26]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9983 x0_1 + 0.1804 x0_111 + 0.0621 x0x0x0_111


In [18]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9983 x0_1 + 0.1830 x0_111 + 0.0565 x0x0x0_111


In [12]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9966 x0_1 + 0.1826 x0_111 + 0.0015 x0x0x0_1 + 0.0019 x0x0_111 + 0.0667 x0x0x0_111


In [15]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9967 x0_1 + 0.1746 x0_111 + 0.0014 x0x0x0_1 + 0.0711 x0x0x0_111


In [46]:
thresh = 8e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.8
(x0)' = 0.9900 x0_1 + 1.4100 x0_111


In [47]:
thresh = 5e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.5
(x0)' = 0.9909 x0_1 + 1.4133 x0_111


In [48]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9932 x0_1 + 0.2852 x0_11 + 1.1621 x0_111 + 0.4053 x0x0x0_111


In [49]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9927 x0_1 + 0.2970 x0_11 + 1.1693 x0_111 + 0.3979 x0x0x0_111


In [50]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9936 x0_1 + 0.2663 x0_11 + 1.1514 x0_111 + 0.0615 x0_1111 + 0.0294 x0x0x0_11 + 0.0090 x0x0_111 + 0.4211 x0x0x0_111 + 0.1124 x0x0x0_1111


## Multifilt (radius 10 smooth)

### Dataset 1

In [ ]:
# fileStr = "2021-11-01T2311_smoothed_rad_10"

# working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
# scaler_val = np.std(working_dat)
# working_dat = working_dat/scaler_val
# working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [ ]:
# thresh = 1e-2
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 3e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-4
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-5
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

### Dataset 2

In [ ]:
# fileStr = "2021-11-01T2321_smoothed_rad_10"

# working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
# scaler_val = np.std(working_dat)
# working_dat = working_dat/scaler_val
# working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [ ]:
# thresh = 1e-2
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 3e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-4
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-5
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

### Dataset 3

In [ ]:
# fileStr = "2021-11-02T0511_smoothed_rad_10"

# working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
# scaler_val = np.std(working_dat)
# working_dat = working_dat/scaler_val
# working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [ ]:
# thresh = 1e-2
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 3e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-4
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-5
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

### Dataset 4

In [ ]:
# fileStr = "2021-11-02T2311_smoothed_rad_10"

# working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
# scaler_val = np.std(working_dat)
# working_dat = working_dat/scaler_val
# working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [ ]:
# thresh = 1e-2
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 3e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-3
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-4
# SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
# thresh = 1e-5
# SINDyImplementEnsemble(working_dat, fileStr,thresh)